In [1]:
from tqdm import tqdm

In [2]:
import os
import pandas as pd
#import numpy as np
import spacy


from io import StringIO
from spacy.tokens import Doc
from csv import QUOTE_NONE

CONLL_COLUMN_NAMES = ["idx", "token", "lemma", "upos", "xpos", "morph", "head", "dep", "enhanced", "misc" ]

class CoNLLReader(object):

    def __init__(self, path):
        self.conll_file = open(path, "r")
        self.nlp = spacy.load("en_core_web_sm")


    def __iter__(self):
        return self

    def __next__(self):
        with StringIO() as buffer:
            metadata = {}
            initial = True
            for line in self.conll_file:
                if line == "\n":
                    if initial:
                        continue
                    else:
                        break

                initial = False

                if line[0] == "#":
                    key, val = line[2:].split("=", 1)
                    key = key.strip()
                    val = val.strip()
                    metadata[key] = val

                else:
                    buffer.write(line)

            buffer.seek(0)
            d = pd.read_csv(buffer, sep="\t", names=CONLL_COLUMN_NAMES, quoting=QUOTE_NONE, dtype="str", keep_default_na=False, na_values=[])
            if len(d) < 1:
                raise StopIteration
            d = d.astype({"head": "int32"})

        try:
            doc = Doc(self.nlp.vocab, words=list(d["token"]))
        except:
            print(d.dtypes)
            print(d)
            print(d["token"])
            raise StopIteration

        for token, lemma in zip(doc, d["lemma"]):
            token.lemma_ = lemma


        for token, tag in zip(doc, d["xpos"]):
            token.tag_ = tag

        for token, dep in zip(doc, d["dep"]):
            token.dep_ = dep

        for token, head in zip(doc, d["head"]):
            if head > 0:
                token.head = doc[head-1]

        return doc, metadata

In [3]:
from change_of_state_better import find_change_of_state
from continuation_of_state_better import find_continuation_of_state

In [4]:
from comparatives import check_comparative
from clefts import check_cleft

In [5]:
from embedded_question import check_sentence_for_embedded_question 
from factives import check_sentence_for_factives, check_sentence_for_quote

In [6]:
from lib.conll_reader import CoNLLReader

In [7]:
dataset_path = "../../datasets/unanswerable_questions_dev.jsonl"    


In [8]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [9]:
import json 

In [42]:
with open(dataset_path) as f:
    for line in f: 
        text = line.rstrip("\n").split(",")[0]
        nlp_text = nlp(text)
        
#         if check_sentence_for_embedded_question(nlp(text))[0]: 
#             print(text)
         
#         if check_sentence_for_factives(nlp_text)[0]: 
#             print(text)
            
#         if check_cleft(nlp_text): 
#             print(text)
     
#         if check_comparative(nlp_text)[0]: 
#             print(text)
            
    
        if find_numeric_determiners(nlp_text): 
            print(text)

who played in the celebrity all star game 2018
who can attend the meeting of both houses of parliament without being a member


In [ ]:
s = "it was rupak that ate the cookies"

In [ ]:
import numpy as np 

In [ ]:
s1 = "it was neha that ate the cookies"

In [ ]:
check_cleft(nlp(s1)) 

In [ ]:
tokens = list(nlp(s1))
#print(tokens)
tokens_str = [str(token) for token in tokens]
#print(tokens_str)
cleft_word = np.intersect1d(tokens_str, ["it"])
#print(cleft_word)
index = tokens_str.index(cleft_word)

cop = tokens[index + 1]
obj = None 
for child in cop.children: 
    #print(child, child.dep_ )
    
    if child.dep_ in ['attr', 'dobj']: 
        obj = child
        break 

In [ ]:
list(obj.children)  

In [ ]:
check_cleft(nlp(s1) ) 

In [ ]:
comp_sent = "Clifford is a bigger dog than Cujo."  # should evaluate to true



In [ ]:
# test sentences
comp_sent = "Is Clifford a bigger dog than Cujo?"  # should evaluate to true
comp_sent2 = "Clifford is a more giant dog than Cujo."  # should evaluate to true
non_comp_sent = "Clifford is a bigger dog than I thought,"  # should evaluate to false
other_comp = "Clifford is bigger of a dog than Cujo."  # should evaluate to true
other_comp2 = "Clifford is bigger a dog than Cujo."  # should evaluate to true, currently doesn't

# displacy.serve(nlp(other_comp2),style='dep')


In [ ]:
check_comparative(nlp(comp_sent))

In [ ]:
check_sentence_for_embedded_question(nlp("I fail to see how you can rationalize rewarding illegality.")) 

In [36]:
s10 = "a microsecond later, images from his exterior sensors didn't snap into focus"

find_change_of_state(nlp(s10))

True

In [24]:
s9 = "Lisa stopped petting Tom's cat"

q = "both kids smiled"
q1 = "all three cat owners that julia spoke to want another cat"
q2 = "both of our fathers were thin"

In [32]:
q3 = "do you see any problem at all because folks on both sides of the aisle have tremendous respect from him?"

In [40]:
s4 = "meaning behind the song whiter shade of pale"

In [13]:
find_change_of_state(nlp(s9))

NameError: name 's9' is not defined

In [15]:
find_continuation_of_state(nlp(s9))

True

In [16]:
from numeric_determiners_better import find_numeric_determiners

In [17]:
find_numeric_determiners(nlp(q))

False

In [38]:
def find_numeric_determiners(sentence):

    numeric_predeterminers =set(["all"])
    special_determiners = set(["both"])
    all_determiners = numeric_predeterminers | special_determiners


    # extract words as list of strings
    words = [t.text.lower() for t in sentence]
    
    #print(words)
    if set(words) & all_determiners:
        for token in sentence:
            include = False
            if token.text.lower() in special_determiners:
                if token.tag_ == "DT" and token.dep_ == "det":
                    include = True
                    return True  
                    
                if not include:
                    for child in token.children:
                        if child.dep_ == "prep" and child.text.lower() == "of":
                            include = True
                            return True 


            if not include and token.text.lower() in numeric_predeterminers:
                        
                if token.tag_ == "DT" and token.dep_ == "det":
                    head_token = token.head

                    if head_token.tag_ == "CD":
                        include = True # all three of the children
                        return True  
                        
                    else :
                        
                        for child in head_token.children:
                            ##print("\t", child, child.dep_, child.tag_)
                            if child.tag_ == "CD" and child.dep_ == "nummod":  ## changed token.dep_ to child.dep_ 
                                #print("here")
                                ##print(sentence)
                                include = True # all three children
                                #print(include)
                                return True 
                
                elif token.tag_ == "DT":
                    for child in token.children:
                        if child.dep_ == "prep" and child.text.lower() == "of":
                            for child_child in child.children:
                                if child_child.dep_ == "pobj":
                                    has_det = False
                                    has_nummod = False
                                    for child_child_child in child_child.children:
                                        if (child_child_child.tag_ == "CD"
                                            and child_child_child.dep_ == "nummod"):
                                            has_nummod = True # all of the three children
                                        if (child_child_child.tag_ == "DT"
                                            and child_child_child.dep_ == "det"):
                                            has_det = True # all of the three children

                                    include = has_det and has_nummod
                                    if include:
                                        #print(sentence)
                                        return True 
                            if include:
                                return True 


    return False




In [48]:
find_numeric_determiners(nlp(s5))

True

In [47]:
s5 = "both channels is administered by the u.s. department of justice"